In [ ]:
from pathlib import Path
import random
import math

import numpy as np
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Random seed set to 42 for reproducibility")

In [ ]:
# Update this path to your local InternVL3 model
model_id = "/path/to/InternVL3-8B"
# Update this path to your test image
imageName = "sample_data/synthetic_invoice_014.png"

print("🔧 Loading InternVL3-8B model for 4x V100 GPUs...")

# Load model
model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval()

# Use DataParallel for 4x V100 multi-GPU utilization
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3])
    model = model.cuda()
    print(f"✅ Using {torch.cuda.device_count()} V100 GPUs with DataParallel")
else:
    model = model.cuda()
    print("✅ Using single GPU")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    use_fast=False
)

print("✅ Model and tokenizer loaded successfully")
if hasattr(model, 'module'):
    print(f"✅ Model replicated across {torch.cuda.device_count()} GPUs")

In [ ]:
# Official InternVL3 image preprocessing (from docs)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Process image and move to GPU
print("🖼️  Processing image...")
pixel_values = load_image(imageName, max_num=12).to(torch.bfloat16).cuda()
print(f"✅ Image processed: {pixel_values.shape}")
print(f"✅ Number of image tiles: {pixel_values.shape[0]}")

# Visual Question Answering - ask a simple question about the image
question = 'How much did Jessica pay?'
# InternVL3 format: <image>\n + question
formatted_question = f'<image>\n{question}'
print(f"❓ Question: {question}")

In [ ]:
# Deterministic generation config with multi-turn support
generation_config = dict(
    max_new_tokens=2000,
    do_sample=False  # Pure greedy decoding for deterministic output
)

# Clear CUDA cache before generation
torch.cuda.empty_cache()

# Generate response using InternVL3 API
print("🤖 Generating response with InternVL3...")
print(f"💾 GPU Memory before generation:")
for i in range(torch.cuda.device_count()):
    allocated = torch.cuda.memory_allocated(i) / 1e9
    reserved = torch.cuda.memory_reserved(i) / 1e9
    print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

try:
    # Access chat() via .module when using DataParallel
    chat_method = model.module.chat if hasattr(model, 'module') else model.chat
    
    response, history = chat_method(
        tokenizer, 
        pixel_values, 
        formatted_question, 
        generation_config,
        history=None,
        return_history=True
    )
    
    print("✅ Response generated successfully!")
    print("\n" + "="*60)
    print("INTERNVL3 RESPONSE:")
    print("="*60)
    print(response)
    print("="*60)
    
except Exception as e:
    print(f"❌ Error during inference: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
finally:
    # Clean up GPU memory
    torch.cuda.empty_cache()

In [ ]:
# Optional: Save response to file
try:
    output_path = Path("internvl3_vqa_output.txt")
    
    with output_path.open("w", encoding="utf-8") as text_file:
        text_file.write(response)
    
    print(f"✅ Response saved to: {output_path}")
    print(f"📄 File size: {output_path.stat().st_size} bytes")
    
except NameError:
    print("❌ Error: 'response' variable not defined.")
    print("💡 Please run the previous cell first to generate the response.")
    
except Exception as e:
    print(f"❌ Error saving file: {e}")